In [ ]:
import ngsolve as ng
from numpy import pi
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
from fiberamp.fiber import ModeSolver

In [ ]:
class SquareFiber(ModeSolver):
    
    def __init__(self, Rsqr=.5, R=1.5, Rout=2.5, ref=0, curve=3):
        
        self.Rsqr = Rsqr
        self.R = R
        self.Rout = Rout
        
        geo = SplineGeometry()
        geo.AddRectangle((-Rsqr,-Rsqr), ((Rsqr,Rsqr)), leftdomain=1, rightdomain=2, bc='square')
        geo.AddCircle((0,0), r=self.R, leftdomain=2, rightdomain=3)
        geo.AddCircle((0,0), r=self.Rout, leftdomain=3, bc='OuterCircle')
        
        geo.SetMaterial(1, 'core')
        geo.SetMaterial(2, 'clad')
        geo.SetMaterial(3, 'Outer')
        
        geo.SetDomainMaxH(1, .1)
        geo.SetDomainMaxH(2, .2)
        geo.SetDomainMaxH(3, 2)
        
        self.geo = geo
        self.mesh = ng.Mesh(geo.GenerateMesh())
        for r in range(ref):
            self.mesh.ngmesh.Refine()
        self.mesh.ngmesh.SetGeometry(self.geo)
        self.mesh.Curve(curve)
        
        self.n0 = 1.45
        self.n_core = 1.48
        self.n_clad = 1.45
        self.wavelength = .5e-6
        self.k = 2*pi/self.wavelength
        self.L = 1.2e-6

        self.N = ng.CoefficientFunction([self.n_core, self.n_clad, self.n0])
        self.V = (self.k * self.L)**2 * (self.n0**2 - self.N**2)
        self.index = self.N

        super().__init__(self.mesh, self.L, self.n0)

In [ ]:
A = SquareFiber(ref=1, curve=10)

In [ ]:
A.mesh.GetBoundaries()

In [ ]:
Draw(A.V, A.mesh)

In [ ]:
z, y, yl, betas, P, hist = A.leakymode(5, ctr=3.3j, rad = .1, alpha=10, npts=4, nspan=2, niterations=10,nrestarts=0 )

In [ ]:
for f in y:
    Draw(1e2*f, A.mesh)

# Product solution after removing interface information?

In [ ]:
f = y[0]

In [ ]:
A.mesh.GetBoundaries()

In [ ]:
gfu = ng.GridFunction(P.fes)

In [ ]:
gfu.Set(f, definedon=A.mesh.Boundaries('square'))

In [ ]:
Draw(1e2*gfu, A.mesh)

In [ ]:
Draw(1e2*(f-gfu), A.mesh)

# Vector

In [ ]:
a = A.sqrZfrom(A.k * A.n_core)

In [ ]:
betas, zs, E, phi, _ = A.leakyvecmodes(ctr=-10.889, rad=.1, p=3, npts=4, nspan=7, 
                                       niterations=20, nrestarts=0, stop_tol=1e-9)

In [ ]:
for f in E:
    Draw(f.real, A.mesh)

In [ ]:
for p in phi:
    Draw(1e2*p, A.mesh)